In [5]:
import SimpleITK as sitk
import os
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
import ipyvolume as ipv
from scipy.ndimage import zoom
import gui
import ipyvolume as ipv
import pandas as pd
%matplotlib notebook

In [2]:
series_reader = sitk.ImageSeriesReader()
cur_paths = series_reader.GetGDCMSeriesFileNames("/home/share/rochester_CT/D175_CT_Scan/D175_Right_Lung_CT_DICOMS/IMediaExport/DICOM/PAT_0000/STD_0000/SER_0001/OBJ_0001")
img = sitk.ReadImage(cur_paths)
img = sitk.RescaleIntensity(img)
temp_lung = sitk.GetArrayViewFromImage(img)
lung_175 = temp_lung.copy()

In [125]:
series_reader = sitk.ImageSeriesReader()
cur_paths = series_reader.GetGDCMSeriesFileNames("/home/share/rochester_CT/Lungmap_D205_anonymized_RightLung/Right Lung/AX BONE .67 - 3")
img = sitk.ReadImage(cur_paths)
temp_lung = sitk.GetArrayViewFromImage(img)
lung_205 = temp_lung

In [48]:
def show_vol_pts(vol, xs, ys, zs, color):
    sphere_size = 4
    ipv.figure()
    ipv.volshow(vol, level=[0.07, 0.22, 0.26], opacity=[0.004, 0.05, 0.1])
    ipv.scatter(zs, ys, xs, marker='sphere', color=color, size=sphere_size)
    ipv.style.use( 'minimal')
    ipv.show()

In [26]:
neighbours_table = pd.read_table("/home/butlera/Projects/HuBMAP/lung-ccf/data/neighbor_points_D205R.tsv")

point = 5

query = neighbours_table.index[point]
query = np.array(query.split(',')).astype(np.int) * 20

In [27]:
query

array([360, 360, 140])

Points to use 
* 1: [680, 380, 180]
* 2: [320, 360, 480]
* 5: [260, 360, 140]

Need to manually adjust bottom sliders
- opacity = 2.40
- brightness = 0.1

In [35]:
x = np.array([680, 320, 260]).astype('f')
y = np.array([380, 360, 360]).astype('f')
z = np.array([180, 480, 140]).astype('f')

In [68]:
show_vol_pts(lung_175, x, y, z, [[0, 0, 1], [0,1,0], [1,0,0]])

In [73]:
def set_view(figure, framenr, fraction):
    ipv.view(90, fraction*360)
ipv.movie('/home/butlera/Projects/HuBMAP/2019-12-05_Hackathon/d175_query_points.gif', set_view, fps=20, frames=160)

Output()

Now plot the neighboring points

In [363]:
neighbours1 = neighbours_table.iloc[1][:]
neighbour1 = np.array([np.array(neighbour.split(',')).astype(np.int) * 20 for neighbour in neighbours1])[0,:]
neighbours2 = neighbours_table.iloc[2][:]
neighbour2 = np.array([np.array(neighbour.split(',')).astype(np.int) * 20 for neighbour in neighbours2])[0,:]
neighbours5 = neighbours_table.iloc[5][:]
neighbour5 = np.array([np.array(neighbour.split(',')).astype(np.int) * 20 for neighbour in neighbours5])[0,:]


In [378]:
all_neighbors = np.concatenate((neighbour1, neighbour2, neighbour5)).reshape(3,3)

In [379]:
x = all_neighbors[:,0].astype('f')
y = all_neighbors[:,1].astype('f')
z = all_neighbors[:,2].astype('f')

In [383]:
c1 = np.array([[0, 0, 1]])
c1 = np.tile(c1,(1,1))
c2 = np.array([[0, 1, 0]])
c2 = np.tile(c2,(1,1))
c3 = np.array([[1, 0, 0]])
c3 = np.tile(c3,(1,1))
colors = np.concatenate((c1, c2, c3), axis = 0)

In [384]:
show_vol_pts(lung_205, x, y, z, colors)

In [388]:
def set_view(figure, framenr, fraction):
    ipv.view(90, fraction*360)
ipv.movie('/home/butlera/Projects/HuBMAP/2019-12-05_Hackathon/d205_neighbor_point_single.gif', set_view, fps=20, frames=160)

Output()

Now for the landmarks

In [345]:
import pandas
landmarks175 = pd.read_csv("/home/butlera/Projects/HuBMAP/D175R_landmarks1.csv")
landmarks175
x = landmarks175.x.to_numpy()
x = -(x - 125)/(0.325520833) 
y = landmarks175.y.to_numpy()
y = -(y + 4)/(0.325520833 ) 
z = landmarks175.z.to_numpy()
z = (z - 17.7099609375)/(0.330017090)


In [346]:
x = x.astype('f')
y = y.astype('f')
z = z.astype('f')
lung_175.shape
print(x)
print(y)
print(z)

[334.99854 404.65    458.99673 128.82433 116.74522 356.3735  353.6118
 587.84564 350.28174 197.99962]
[394.45096 315.51898 519.3953  330.14783 308.0018  207.86382 241.06598
 324.10828 507.31622 555.86   ]
[8.2085760e+02 5.4813232e+02 6.6095380e+02 6.9160172e+01 1.7516075e+02
 3.5504840e+02 3.6825073e+02 1.6159887e+01 1.6071610e-01 4.8843845e+02]


In [370]:
show_vol_pts(lung_175, z, y, x, 'red')

In [362]:
def set_view(figure, framenr, fraction):
    ipv.view(90, fraction*360)
ipv.movie('/home/butlera/Projects/HuBMAP/2019-12-05_Hackathon/d175R_landmarks.gif', set_view, fps=20, frames=160)

Output()